# Ideas
- would be good to get going in leaflet 

In [1]:
from helper_functions import *
from osgeo import gdal
gdal.UseExceptions()
import numpy as np
import pandas as pd

In [2]:
#set paths
fulres_4326_path = r"intermediate_data/fulres_4326.tif"
lowres_4326_path = r"intermediate_data/lowres_4326.tif"
output_nc_path = r"output_data/output_nc.nc"
input_tif_5070_path = r"input_data/1992_2015/1992.tif"
template_path = r"template_data/dust_emissions_05.20210906.nc"
scaling_factor = 0.1

# take peace out of input file
- get slice
- take measurements
    - measure total area
    - measure salt

In [3]:
# open file
xr_array = rioxarray.open_rasterio(input_tif_5070_path)
xr_array = xr_array.where(xr_array != xr_array._FillValue) #tiff files have a _FillValue to hold none values
display(xr_array)
xr_array.plot()

<xarray.DataArray (band: 1, y: 3133, x: 4836)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.493e+06 -2.492e+06 ... 2.341e+06 2.342e+06
  * y            (y) float64 3.31e+06 3.309e+06 ... 1.788e+05 1.778e+05
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:           Area
    RepresentationType:      ATHEMATIC
    STATISTICS_COVARIANCES:  135369674.2310497
    STATISTICS_MAXIMUM:      755652.375
    STATISTICS_MEAN:         3186.8445232266
    STATISTICS_MINIMUM:      0
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       11634.847409014
    _FillValue:              -3.4028235e+38
    scale_factor:            1.0
    add_offset:              0.0

In [4]:
# slice input file in memory (I'm thinking at y values)
slices = np.arange(0, int(3.5e6), int(0.5e6))
print(slices)
xr_array = xr_array.where(xr_array.y>slices[0]).where(xr_array.y<slices[1])
display(xr_array)
xr_array.plot()

[      0  500000 1000000 1500000 2000000 2500000 3000000]


<xarray.DataArray (band: 1, y: 3133, x: 4836)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.493e+06 -2.492e+06 ... 2.341e+06 2.342e+06
  * y            (y) float64 3.31e+06 3.309e+06 ... 1.788e+05 1.778e+05
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:           Area
    RepresentationType:      ATHEMATIC
    STATISTICS_COVARIANCES:  135369674.2310497
    STATISTICS_MAXIMUM:      755652.375
    STATISTICS_MEAN:         3186.8445232266
    STATISTICS_MINIMUM:      0
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       11634.847409014
    _FillValue:              -3.4028235e+38
    scale_factor:            1.0
    add_offset:              0.0

Total in slice = 565116
Lowest x = -2492545
Highest x = 2342455
Lowest y = 177785
Highest y = 3309785


,Total_in_slice,Lowest_x,Highest_x,Lowest_y,Highest_y
0,565116,-2492545,2342455,177785,3309785


: 

In [ ]:
# take stats salt, area, furthest left (lowest longitude > -180), furthest right (highest longitude < 180), lowest lat > -90, highest lat < 90
print(f"Total in slice = {int(xr_array.sum())}")
print(f"Lowest x = {int(xr_array.x.min())}")
print(f"Highest x = {int(xr_array.x.max())}")
print(f"Lowest y = {int(xr_array.y.min())}")
print(f"Highest y = {int(xr_array.y.max())}")

stats = pd.DataFrame({
    "Total_in_slice":int(xr_array.sum()),
    "Lowest_x":int(xr_array.x.min()),
    "Highest_x":int(xr_array.x.max()),
    "Lowest_y":int(xr_array.y.min()),
    "Highest_y":int(xr_array.y.max())
    }, index = [0])

display(stats)

In [ ]:
# re-save files and stats as slices

# run regrinding  method on piece

# measure output files 
- measure total area
- measure total salt